In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.signal import medfilt
from sklearn.decomposition import PCA
from tqdm import tqdm

%matplotlib widget

In [ ]:
rand_data = np.random.randn(50, 2000)
for i in tqdm(range(rand_data.shape[1])):
    rand_data[:, i] = medfilt(rand_data[:, i], 15)

In [ ]:
corrmat = np.corrcoef(rand_data.T)  # .min()

In [ ]:
filt = np.argwhere(corrmat.min(1) < -0.65).flatten()

In [ ]:
pca = PCA(n_components=5)
pca.fit(rand_data[:, filt].T)
pca_decom = pca.transform(rand_data[:, filt].T)

In [ ]:
plt.figure()
plt.scatter(pca_decom[:, 0], pca_decom[:, 1])

In [ ]:
import colorspacious

pca_proj = np.random.randn(1000, 2)  # fake roi projections

angles = np.angle(pca_proj[:, 0] + 1j * pca_proj[:, 1])  # roi angles
amplitudes = np.absolute(
    pca_proj[:, 0] + 1j * pca_proj[:, 1]
)  # roi distance from center

In [ ]:
hueshift = (0,)  # rotate to rotate color wheel
amp_percentile = (90,)  # percentile at which to reach saturation
maxsat = (50,)  # maximum saturation reached
lightness_min = (90,)  #  minimum lightness reached
lightness_delta = (-80,)  # maximum lightness at center

output_lch = np.empty((len(angles), 3))
maxamp = np.percentile(amplitudes, amp_percentile)

output_lch[:, 0] = (
    lightness_min + (np.clip(amplitudes / maxamp, 0, 1)) * lightness_delta
)
output_lch[:, 1] = (np.clip(amplitudes / maxamp, 0, 1)) * maxsat
output_lch[:, 2] = (angles + hueshift) * 180 / np.pi

output_rgb = np.clip(colorspacious.cspace_convert(output_lch, "JCh", "sRGB1"), 0, 1)

In [ ]:
plt.figure(figsize=(3, 3))
plt.scatter(pca_proj[:, 0], pca_proj[:, 1], c=output_rgb)

In [ ]:
output_lch[0, :]